In [1]:
import json
import os
import re
from utils import *

In [2]:
track_uri_to_id, artist_uri_to_id, _ = get_training_set_info() 

In [3]:
valid_playlists = list() #all playlists in the validation set

In [4]:
def process_playlist_for_validation(playlist, valid_playlists):
    tracks  = []
    artists = []
    for track in playlist['tracks']:
        track_uri  = track['track_uri'].split(':')[2]
        artist_uri = track['artist_uri'].split(':')[2]
        # not consider tracks that did not appear in the training set.
        if track_uri in track_uri_to_id:
            track_id = track_uri_to_id[track_uri]
            tracks.append(track_id)
        if artist_uri in artist_uri_to_id:
            artist_id = artist_uri_to_id[artist_uri]
            artists.append(artist_id)
            
    #get vector representation of playlist name
    name = normalize_name(playlist['name'])
    title_char_indices = title_to_indices(name, MAX_TITLE_LEN)

    valid_playlists.append([tracks, artists, title_char_indices])

In [5]:
filenames = os.listdir(valid_fullpaths)
for filename in sorted(filenames):
     if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        fullpath = os.sep.join((valid_fullpaths, filename))
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        
        #add playlist to validation set
        for playlist in mpd_slice["playlists"]:
            process_playlist_for_validation(playlist, valid_playlists)

#save validation data
file_data              = {}
file_data['playlists'] = valid_playlists
if not os.path.isdir(valid_data_dir):
    os.mkdir(valid_data_dir)
with open(valid_json, 'w') as make_file:
    json.dump(file_data, make_file, indent="\t")
print("Number of playlists: %d" % len(valid_playlists))

Number of playlists: 50000
